In [76]:
import requests

In [105]:
def get_etf_list():
    URL = 'https://finance.naver.com/api/sise/etfItemList.nhn'
    response = requests.get(URL)
    etf_item_list = response.json().get('result').get('etfItemList')
    print(f'etf_item_list : {len(etf_item_list)}')
    return etf_item_list

In [79]:
import pandas as pd

In [106]:
etf_item_df = pd.DataFrame(get_etf_list())
etf_item_df.head()

etf_item_list : 805


,itemcode,etfTabCode,itemname,nowVal,risefall,changeVal,changeRate,nav,threeMonthEarnRate,quant,amonut,marketSum
0,357870,6,TIGER CD금리투자KIS(합성),53500,2,5,0.01,53499.0,0.9339,154911,8287,68317
1,459580,6,KODEX CD금리액티브(합성),1020715,2,335,0.03,1020712.0,0.9088,301613,307860,61531
2,069500,1,KODEX 200,34365,2,195,0.57,34522.0,0.3819,3814161,131760,59950
3,449170,6,TIGER KOFR금리액티브(합성),103755,2,10,0.01,103756.0,0.9241,17720,1838,50402
4,423160,6,KODEX KOFR금리액티브(합성),105210,2,30,0.03,105198.0,0.9211,263889,27763,47108


In [113]:
kwds = ['액티브','TR','배당','금리','단기','혼합','MSCI','인도']

def filter_etf_item(df, kwds):
    return df.loc[
        (df.etfTabCode != 1)
        &(df.quant > df.quant.median())
        &(df.marketSum > df.marketSum.median() * 2)
        &~df.itemname.str.contains('|'.join(kwds))
        &(~df.itemname.str.contains('합성')
            |df.itemname.str.contains('인버스'))]\
        .iloc[:, [0, 2]]\
        .reset_index(drop=True)

etf_item_filterd = filter_etf_item(etf_item_df, kwds)
etf_item_filterd.head()

,itemcode,itemname
0,133690,TIGER 미국나스닥100
1,360750,TIGER 미국S&P500
2,122630,KODEX 레버리지
3,371460,TIGER 차이나전기차SOLACTIVE
4,381170,TIGER 미국테크TOP10 INDXX


In [114]:
def get_unique_etf_item(df):
    df_ = df.copy()
    df_['sector'] = df_.itemname.replace(
    '(^\D+ )|(\(H\)|Fn|iSelect|SOLACTIVE|INDXX|선물|\s|플러스|Plus|PLUS)',
    '', regex=True)
    return df_.groupby('sector')\
        .first()[['itemcode', 'itemname']]\
        .set_index('itemcode').sort_index()

etf_item_unique = get_unique_etf_item(etf_item_filterd)
etf_item_unique.head()

,itemname
itemcode,
091160,KODEX 반도체
091170,KODEX 은행
091180,KODEX 자동차
102780,KODEX 삼성그룹
102970,KODEX 증권


In [115]:
import ast

In [149]:
def get_price(symbol):
    URL = 'https://api.finance.naver.com/siseJson.naver'
    params = dict(symbol=symbol,
                  requestType=1,
                  startTime='20230101',
                  endTime='20991231',
                  timeframe='day')
    response = requests.get(URL, params=params)
    literal = response.text.replace('\n', '').replace('\t', '')
    data = ast.literal_eval(literal)
    df = pd.DataFrame(data[1:], columns=data[0])
    return (symbol, df.drop(columns=['시가', '거래량', '외국인소진율']))

symbol, price =  get_price('357870')
price.head()

,날짜,고가,저가,종가
0,20230102,51700,51695,51700
1,20230103,51705,51700,51700
2,20230104,51710,51705,51705
3,20230105,51725,51720,51720
4,20230106,51730,51725,51725


In [140]:
from multiprocessing import cpu_count
from concurrent.futures import ThreadPoolExecutor
import time

In [147]:
cpu_num = cpu_count()
print(f'cpu_num : {cpu_num} → worker_num : {cpu_num * 2}')

t = time.time()

# with ThreadPoolExecutor() as executor:
with ThreadPoolExecutor(max_workers=cpu_num * 2) as executor:
    results = executor.map(get_price, etf_item_unique.index)

print(f'process time : {time.time() - t:.3f}s')

prices = {}
for symbol, price in results:
    prices[symbol] = price

prices.keys()

cpu_num : 2 → worker_num : 4
process time : 17.677s


dict_keys(['091160', '091170', '091180', '102780', '102970', '108450', '114260', '114800', '122630', '132030', '133690', '138540', '139230', '139250', '139260', '139280', '143860', '144600', '148070', '150460', '157490', '192090', '203780', '204450', '213610', '227540', '228790', '228800', '228810', '233740', '241180', '244580', '245340', '245360', '251340', '252670', '261220', '261240', '261260', '261270', '267770', '271050', '290130', '292150', '292560', '298770', '300640', '302190', '304660', '305080', '305540', '305720', '309230', '314250', '325010', '326240', '329200', '360750', '364960', '364970', '364980', '365000', '367760', '367770', '371160', '371450', '371460', '371470', '381170', '381180', '385560', '390390', '394660', '395160', '395170', '395270', '395290', '396500', '400570', '401170', '411060', '412570', '441540', '446770', '449450', '455850', '455860', '461950', '462010', '462330', '464930', '465350', '465580', '471760', '471990'])